# Auriez-vous survécu au naufrage du titanic?

- Le but de cet exercice est de développer un modèle de prédiction de type kNN sur le jeu de données des survivants du Titanic.
- Ce modèle devra être développé et optimisé suivant la méthodologie présentée à l'exercice 1 (iris)
- Il pourra alors être utilisé pour déterminer qui d'entre nous aurait eu le plus de chance de survivre au naufrage du Titanic
- On pourra préciser les probabilités de survie à l'aide de la fonction predict_proba

### 1) Chargement et préparation des données

In [8]:
# On commence par importer nos bibliothèques habituelles

import sklearn
from sklearn import datasets
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

# a)

In [31]:
# Import du fichier (en local)

fichier = pd.read_excel ('titanic3.xls')


# Aperçu

fichier.head ()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


# b)

In [32]:
# Modifier les données de la manière suivante :


# on ne garde que ces 4 caractéristiques : 'survived','pclass','sex','age'

data = fichier.drop (
[
    'name',
    'sibsp',
    'parch',
    'ticket',
    'fare',
    'cabin',
    'embarked',
    'boat',
    'body',
    'home.dest'
], axis = 1)


# on élimine les données incomplètes

data.dropna (axis = 0, inplace = True)


# on remplace male par 0 et female par 1

data.replace (to_replace = 'male', value = {'sex': 0}, inplace = True)
data.replace (to_replace = 'female', value = {'sex': 1}, inplace = True)


# Résultat

data.head ()

,pclass,survived,sex,age
0,1,1,1,29.0000
1,1,1,0,0.9167
2,1,0,1,2.0000
3,1,0,0,30.0000
4,1,0,1,25.0000


# c)



In [33]:
data.describe ()

,pclass,survived,sex,age
count,1046.000000,1046.000000,1046.000000,1046.000000
mean,2.207457,0.408222,0.370937,29.881135
std,0.841497,0.491740,0.483287,14.413500
min,1.000000,0.000000,0.000000,0.166700
25%,1.000000,0.000000,0.000000,21.000000
50%,2.000000,0.000000,0.000000,28.000000
75%,3.000000,1.000000,1.000000,39.000000
max,3.000000,1.000000,1.000000,80.000000


In [61]:
# la colonne survived sera notre Y et class, sexe et âges nos caractéristiques X


# Nombre de points voisins

global k
k = 11


# Nombre de données d'entraînement

global t
t = 800


# L'on normalise les lois que suivent variables ...

def normaliser (individu):
    
    # (valeur - moyenne) / ecart-type
    pclass = (individu[0] - 2.207457) / 0.841497
    sex = (individu[1] - 0.370937) / 0.483287
    age = (individu[2] - 29.881135) / 14.413500
    
    return (pclass, sex, age)


# ... pour pouvoir effectuer la distance entre 2 vecteurs de celles-ci.

def distance (individuA, individuB):
    
    donneeA = normaliser (individuA)
    donneeB = normaliser (individuB)
    
    ecartPclass = donneeA[0] - donneeB[0]
    ecartSex = donneeA[1] - donneeB[1]
    ecartAge = donneeA[2] - donneeB[2]
    
    # Calcul de la norme euclidienne
    return (ecartPclass ** 2 + ecartSex ** 2 + ecartAge ** 2) ** 0.5


# Fonction qui détermine la survie d'une personne
# dont la classe, le genre et l'âge sont fournies

def reponse (individu):
    
    survivre = []
    # Le slice [: t] représente les données d'entraînement
    for ligne in data.index[: t]:
        
        connu = (
            data.loc[ligne, 'pclass'],
            data.loc[ligne, 'sex'],
            data.loc[ligne, 'age']
        )
        
        survivre.append ((distance (connu, individu), data.loc[ligne, 'survived']))
    
    # Tri des images en fonction de la distance avec l'individu de départ
    proches = sorted (survivre, key = lambda x: x[0])[: k]
    images = [p[1] for p in proches]
    
    moyenne = sum (images) / k
    
    return round (moyenne)


def tests ():
    
    succes = 0
    echecs = 0
    # Le slice [t:] représente les données d'évaluation
    for ligne in data.index[t:]:
        
        personne = (
            data.loc[ligne, 'pclass'],
            data.loc[ligne, 'sex'],
            data.loc[ligne, 'age']
        )
        
        if reponse (personne) == data.loc[ligne, 'survived']:
            succes += 1
            
        else:
            echecs += 1
            
    print ("Succès :", succes)
    print ("Échecs :", echecs)
    
    
tests ()

Succès : 184
Échecs : 62
